In [1]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

#主要輸入接收新聞標題本身，即一個整數序列（每個整數編碼一個詞）。
#這些整數在1 到10,000 之間（10,000 個詞的詞彙表），且序列長度為100 個詞
#宣告一個 NAME 去定義Input
INPUT = Input(shape=(100,), dtype='int32', name='INPUT')


# Embedding 層將輸入序列編碼為一個稠密向量的序列，
# 每個向量維度為 512。
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(INPUT)

# LSTM 層把向量序列轉換成單個向量，
# 它包含整個序列的上下文信息
lstm_out = LSTM(32)(x)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
#插入輔助損失，使得即使在模型主損失很高的情況下，LSTM 層和Embedding 層都能被平穩地訓練
NEWS_out = Dense(1, activation='sigmoid', name='NEWS_out')(lstm_out)

In [3]:
#輔助輸入數據與LSTM 層的輸出連接起來，輸入到模型
import keras
NEWS_in = Input(shape=(5,), name='NEWS_in')
x = keras.layers.concatenate([lstm_out, NEWS_in])


# 堆疊多個全連接網路層
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
#作業解答: 新增兩層
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(8, activation='relu')(x)

# 最後添加主要的邏輯回歸層
OUTPUT = Dense(1, activation='sigmoid', name='OUTPUT')(x)

In [4]:
# 宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
model = Model(inputs=[INPUT, NEWS_in], outputs=[OUTPUT, NEWS_out])

In [5]:
model.compile(optimizer='rmsprop',
              loss={'OUTPUT': 'binary_crossentropy', 'NEWS_out': 'binary_crossentropy'},
              loss_weights={'OUTPUT': 1., 'NEWS_out': 0.2})

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
INPUT (InputLayer)              (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     5120000     INPUT[0][0]                      
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       embedding_1[0][0]                
__________________________________________________________________________________________________
NEWS_in (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
concatenat